# Alchemy API



Alchemy API is our Swiss Army Knife for natural lanugage processing. It offers a number of NLP microservices including named entity recognition, concept extraction and targetted sentiment analysis.

Let's start off by setting up Alchemy API. You need to go into bluemix and [create an instance of alchemy](https://console.ng.bluemix.net/catalog/services/alchemyapi/).

Then you'll need to find your alchemy API key and paste it into the following snippet and run.

In [4]:
API_KEY = "<your_key_here>"

with open("api_key.txt","w") as f:
    f.write(API_KEY)
    print("saved api key...")

saved api key...


Once you've got an API key we can start using the API itself. We will start off with overall document sentiment.

## Sentiment Analysis

The sentiment analysis endpoint allows you to retrieve an overall score for a document's sentiment. Here we look at an article about [David Bowie's death](http://www.bbc.co.uk/news/entertainment-arts-35278872) to find out the overall tone of the document.

In [5]:
from alchemyapi import AlchemyAPI

alchemyapi = AlchemyAPI()

print (alchemyapi.sentiment('url','http://www.bbc.co.uk/news/entertainment-arts-35278872')['docSentiment'])

{'score': '0.288876', 'mixed': '1', 'type': 'positive'}


As expected this write up is very mixed although comes out as positive overall. This is not particularly useful when the subject is so melancholy and multifaceted. Let's investigate some of the entities and concepts in the document and then use targetted emotion to specifically find out the author's opinion on these matters.

## Concept extraction



In [14]:
ents = alchemyapi.entities('url','http://www.bbc.co.uk/news/entertainment-arts-35278872')
for  e in ents['entities']:
    print (e['text'], "|", e['type'], "|", e['relevance'])

David Bowie | Person | 0.786537
Sir Paul McCartney | Person | 0.213444
UK | Country | 0.206861
Blackstar | Person | 0.197027
David | Person | 0.196996
New York | City | 0.192178
Twitter | Company | 0.189424
Facebook | Company | 0.184585
BBC | Company | 0.172416
producer | JobTitle | 0.171418
Iggy Pop | Person | 0.169573
New York | City | 0.166362
Tony Visconti | Person | 0.163646
Brixton | City | 0.162836
Elvis Presley | Person | 0.160523
cancer | HealthCondition | 0.159453
social media | FieldTerminology | 0.159094
London | City | 0.158075
Brian May | Person | 0.156929
Duncan Jones | Person | 0.156329
Midge Ure | Person | 0.156259
Ziggy | Person | 0.155002
Brian Eno | Person | 0.152916
David Jones | Person | 0.15165
Panthers | Organization | 0.151284
Ricky Gervais | Person | 0.150862
BBC Arts | PrintMedia | 0.150781
Davy Jones | Person | 0.146958
Chris Hadfield | Person | 0.146879
director | JobTitle | 0.146746
Monkees | Organization | 0.14508
Merce Cunningham | Person | 0.145074
Merc

There are a number of entities covered in this write up. Lots of celebrities chimed in with something to say about Bowie passing away. We can examine how these people are regarded.

In [16]:
for e in [e for e in ents['entities'] if e['type'] == "Person"]:
    sent = alchemyapi.sentiment_targeted('url','http://www.bbc.co.uk/news/entertainment-arts-35278872',e['text'])
    if 'docSentiment' in sent:
        print ( e['text'], sent['docSentiment']['type'], 
               ( sent['docSentiment']['score'] if 'score' in sent['docSentiment'] else '') )

David Bowie positive 0.11641
Sir Paul McCartney neutral 
Blackstar negative -0.0118115
David positive 0.257304
Iggy Pop positive 0.342849
Tony Visconti positive 0.423158
Elvis Presley positive 0.477191
Brian May neutral 
Duncan Jones neutral 
Midge Ure neutral 
Ziggy negative -0.0118972
Brian Eno neutral 
David Jones neutral 
Ricky Gervais negative -0.576202
Davy Jones positive 0.665917
Chris Hadfield neutral 
Merce Cunningham positive 0.734231
Nicolas Roeg neutral 
Yoko Ono neutral 
Lazarus neutral 
Madonna negative -0.259528
John Lennon neutral 
Mark Savage negative -0.340568
Andy Warhol positive 0.598561
Michael Eavis neutral 


In the article, David Bowie is, unsurprisingly, regarded as positive. There is a mix of positive and negative emotions around many of the people in the article.


## Social media Analysis
Similar analysis can easily be applied to twitter. 

The following code runs using the vanilla python twitter module which can be installed via `pip install twitter`.

You will also need to [sign up for a developer account and get access keys](https://apps.twitter.com/) to paste into the variables below.

In [18]:
from twitter import Twitter, OAuth

con_secret = "<CONSUMER_TOKEN>"
con_secret_key = "<CONSUMER_SECRET>"
token = "<TOKEN>"
token_key = "<TOKEN_KEY>"

t = Twitter(auth=OAuth(token, token_key, con_secret, con_secret_key))

Once we've initialised our twitter object we can search for tweets about Bowie and get entities mentioned in tweets.

In [19]:
tweets = t.search.tweets(q="#bowie")

for tweet in tweets['statuses']:
    print (tweet['user']['name'])
    print (tweet['text'])
    print (alchemyapi.entities('text', tweet['text']))

Parking Kong
RT @lecoolbarcelona: Llegamos! #newsletter con los mejores planes de #Barcelona y #foto de @PilgrimKhan https://t.co/clg6Bd3Sdz #Bowie http…
{'url': 'http://www.bbc.co.uk/news/entertainment-arts-35278872', 'entities': [], 'usage': 'By accessing AlchemyAPI or using information generated by AlchemyAPI, you are agreeing to be bound by the AlchemyAPI Terms of Use: http://www.alchemyapi.com/company/terms.html', 'status': 'OK', 'language': 'spanish'}
Adam Collins
#Bowie @ Ding Dong Lounge https://t.co/SOe3QUn8J8
{'url': 'http://www.bbc.co.uk/news/entertainment-arts-35278872', 'entities': [], 'usage': 'By accessing AlchemyAPI or using information generated by AlchemyAPI, you are agreeing to be bound by the AlchemyAPI Terms of Use: http://www.alchemyapi.com/company/terms.html', 'status': 'OK', 'language': 'german'}
Supertriglia
Perché le aziende dovrebbero riascoltare David #Bowie https://t.co/7XrmkRm695 https://t.co/WTl5J8BWNC
{'url': 'http://www.bbc.co.uk/news/entertainment-arts

As you can see, Alchemy identifies people mentioned as well as hashtags and twitter handles. We can also target sentiment within tweets.

In [20]:
tweets = t.search.tweets(q="#bowie")

for tweet in tweets['statuses']:
    print (tweet['user']['name'])
    print (tweet['text'])
    entities = alchemyapi.entities('text', tweet['text'])
    for e in entities['entities']:
        sent = alchemyapi.sentiment_targeted('text',tweet['text'],e['text'])
        if 'docSentiment' in sent:
            print ( e['text'], e['type'], sent['docSentiment']['type'], 
                   ( sent['docSentiment']['score'] if 'score' in sent['docSentiment'] else '') )

Graham Watkins
RT @ADonnellyWriter: #Bowie tribute...? https://t.co/aw7KnwFcJ5

@freeboostpromo #bookboost
@ebookbump 
#iartg #asmsg
#RT
Bowie Person positive 0.435633
JP Downing
RT @crowdermusic: I think I need to re-watch Labyrinth because I think it may actually have been a maze.  #bowie #maze vs  #labryinth
Fito Kuadra
Space oddity by Rockdeluxe #Bowie https://t.co/ylIoM15Vas
Tim Falconer
RT @andykat2: So far #Bowie's death has led to an outpouring of people talking about themselves. Yay, us. We're really great.
Bowie Person negative -0.607994
Mad Witch
Awwwww #Bowie on extras 😍😍😍😍😍😍
Miss Fletcher
@sonolucadini Non ho ancora trovato Vanity in edicola e ciò mi fa pensare che arriverà qualcosa di speciale su #Bowie, grazie in anticipo.
rotekatzentatze
RT @Ms_Golightly: Happy Birthday, Mister #Bowie! https://t.co/S0Ku88N5WD
Free eBook Promotion
RT @ADonnellyWriter: #Bowie tribute...? https://t.co/aw7KnwFcJ5

@freeboostpromo #bookboost
@ebookbump 
#iartg #asmsg
#RT
Bowie Person positiv

As you can see there is a mixed response to Bowie's death on twitter. Most people talking about how him in a positive way, a few decrying him.